# Setup

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import re
import time
import pickle

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter

from sklearn import metrics
from sklearn.metrics import classification_report, f1_score

In [18]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
dataset = pickle.load(open('conll_graph.pickle', 'rb'))
print(', '.join([split + f' : {len(dataset[split])}' for split in dataset]))

train : 178610, validation : 44900, test : 40760


In [5]:
dataset['train'][2]

{'word': ['german'],
 'label': 'MISC',
 'gt_label': 'B-MISC',
 'surface': 'German',
 'pos': ['<JJ>'],
 'chunk': ['<NP>'],
 'classes': ['<GEOREGION>', '<NAME>', '<GIVEN NAME>', '<FAMILY NAME>'],
 'extra': ['<CAPITALIZED>'],
 'left_context': ['eu', 'rejects'],
 'right_context': ['call', 'to', 'boycott', 'british', 'lamb']}

In [6]:
vocabulary = pickle.load(open('vocabulary.pickle', 'rb'))
print(', '.join([key + f' : {len(vocabulary[key])}' for key in vocabulary]))

word : 18823, chunk : 11, pos : 25, classes : 14, extra : 3


In [7]:
labels = pickle.load(open('labels.pickle', 'rb'))
label2id = {l: i for i, l in enumerate(labels)}
print(label2id)

{'LOC': 0, 'MISC': 1, 'O': 2, 'ORG': 3, 'PER': 4}


In [8]:
voc2id = {}
for key in vocabulary:
    voc2id[key] = {l: i for i, l in enumerate(vocabulary[key])}
print(voc2id['word']['ismail'], voc2id['extra']['<ACRONYM>'], )

8532 0


# Dataset

In [9]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, split, voc2id=voc2id, label2id=label2id, context='all'):
        X = []
        Y = []
        
        for doc in tqdm(dataset[split], desc=split.upper()):
            # Create the graph one-hot embedding
            onehot = {}
            for key in voc2id:
                onehot[key] = np.zeros(len(voc2id[key]))
                for v in doc[key]:
                    onehot[key][voc2id[key][v]] = 1.
            
            onehot['left'] = np.zeros(len(voc2id['word']))
            onehot['right'] = np.zeros(len(voc2id['word']))
            
            if context == 'all':
                context = max(len(doc['left_context']), len(doc['right_context']))
            
            for w in doc['left_context'][-context:]:
                onehot['left'][voc2id['word'][w]] = 1.
            for w in doc['right_context'][:context]:
                onehot['right'][voc2id['word'][w]] = 1.
            
            doc_embedding = np.concatenate([onehot['word'], onehot['left'], onehot['right'],
                                            onehot['pos'], onehot['chunk'], onehot['extra'], onehot['classes']])
            X.append(doc_embedding)
            Y.append(label2id[doc['label']])
        
        self.X = X
        self.Y = Y
        self.X_len = len(X)
        self.labels = sorted(label2id.keys())
        self.label2id = label2id
        self.voc2id = voc2id

    def __len__(self):
        return self.X_len

    def __getitem__(self, index):
        x = torch.tensor(self.X[index])
        y = torch.tensor(self.Y[index])
        
        return x, y

    def labels(self):
        return self.labels
    
    def voc(self, key):
        return self.voc2id[key]
    
    def Y(self):
        return self.Y

In [10]:
train_set = Dataset(dataset, 'train', context=3)
dev_set = Dataset(dataset, 'validation', context=3)
test_set = Dataset(dataset, 'test', context=3)

In [35]:
batch_size = 64
num_workers = 4

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev_set, batch_size=batch_size, num_workers=num_workers, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers, shuffle=False)

In [36]:
input_dim = 0
for local_features, local_labels in train_loader:
    input_dim = local_features.shape[1]
    print(local_features.shape)
    print(local_labels.shape)
    break

torch.Size([64, 56522])
torch.Size([64])


In [37]:
input_dim

56522

In [38]:
training_counter = Counter(train_set.Y)
print(training_counter)

Counter({2: 144631, 4: 11124, 3: 9984, 0: 8288, 1: 4583})


In [39]:
labels

['LOC', 'MISC', 'O', 'ORG', 'PER']

# The Model

In [40]:
def backprop(batch_X, batch_Y, model, optimizer, loss_fn):
    Y_hat = model(batch_X)
    loss = loss_fn(Y_hat, batch_Y)
    loss.backward()
    optimizer.step()
    
    return loss.item()

class FeedForwardNetwork(nn.Module):
    def __init__(self, input_dim=input_dim, hidden_dim=256, output_dim=5, dropout_rate=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fch = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

        # extra layers layers
        self.dropout = nn.Dropout(p=dropout_rate)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fch(x)
        x = self.relu(x)
        x = self.dropout(x)
        logits = self.fc2(x)

        return logits

In [41]:
ffnet = FeedForwardNetwork(dropout_rate=0.2).to(device)

In [42]:
logs = {'loss/train': {}, 'dev': {}}
writer = SummaryWriter(comment='xp4-onehot-allfeat-win3-wei2-lr1e3-mom0.9-wd5e4-hd256-dr0.2-bs64', log_dir=None,)

In [43]:
label_counter   = Counter(train_set.Y)
labels_freqs    = [label_counter[label] / sum(label_counter.values()) for label in range(len(labels))]
labels_weights1 = [min(label_counter.values()) / label_counter[label] for label in range(len(labels))]
labels_weights2 = [np.sqrt(min(label_counter.values())) / np.sqrt(label_counter[label]) for label in range(len(labels))]

weights = torch.Tensor(labels_weights2).to(device)
print(weights)

tensor([0.7436, 1.0000, 0.1780, 0.6775, 0.6419], device='cuda:0')


In [44]:
optimizer_params = {'lr': 1e-3, 
                    'momentum': 0.9, 
                    'weight_decay': 5e-4,
                   }

log_interval = int(len(train_loader) / 5)

loss_fn = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.SGD(ffnet.parameters(), **optimizer_params)

In [45]:
%%time
max_epochs = 15

for epoch in range(len(logs['loss/train']), len(logs['loss/train']) + max_epochs):
    
    # Training
    ffnet.train()
    print('Epoch', epoch)
    logs['loss/train'][epoch] = []
    writer.add_scalar("Learning_rate", optimizer_params['lr'], epoch)

    for batch, (batch_X, batch_Y) in enumerate(tqdm(train_loader)):
        # tranfer to GPU
        batch_X, batch_Y = batch_X.float().to(device), batch_Y.to(device)
        optimizer.zero_grad()
        l = backprop(batch_X, batch_Y, ffnet, optimizer, loss_fn)
        logs['loss/train'][epoch].append(l)
        
        if batch % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch * len(batch_X), len(train_loader.dataset),
                100. * batch / len(train_loader), l))
    
    logs['loss/train'][epoch] = np.mean(logs['loss/train'][epoch])
    writer.add_scalar("Loss/train", logs['loss/train'][epoch], epoch)
    print(f'Average loss on epoch {epoch}: {logs["loss/train"][epoch]}')
    
    # Validation
    ffnet.eval()
    with torch.no_grad():
        preds = []
        gt = []
        for batch, (batch_X, batch_Y) in enumerate(tqdm(dev_loader)):
            # Transfer to GPU
            batch_X = batch_X.float().to(device)
            output = nn.Softmax(dim=1)(ffnet(batch_X))
            preds.append(output.cpu())
            gt.append(batch_Y)

        all_out = [np.argmax(l) for batch in preds for l in batch.numpy()]
        all_gt  = [l for batch in gt for l in batch.numpy()]

        print(classification_report(all_out, all_gt, digits=4))

        micro_F1 = metrics.f1_score(all_gt, all_out, average='micro')
        macro_F1 = metrics.f1_score(all_gt, all_out, average='macro')
        weighted_F1 = metrics.f1_score(all_gt, all_out, average='weighted')
        writer.add_scalar("micro_F1/dev", micro_F1, epoch)
        writer.add_scalar("macro_F1/dev", macro_F1, epoch)
        writer.add_scalar("weighted_F1/dev", weighted_F1, epoch)
        logs['dev'][epoch] = (micro_F1, weighted_F1, macro_F1, (all_gt, all_out))

Epoch 0


Train Epoch: 0 [0/178610 (0%)]	Loss: 1.599002
Train Epoch: 0 [35712/178610 (20%)]	Loss: 1.386837
Train Epoch: 0 [71424/178610 (40%)]	Loss: 0.978038
Train Epoch: 0 [107136/178610 (60%)]	Loss: 0.891928
Train Epoch: 0 [142848/178610 (80%)]	Loss: 1.110513
Train Epoch: 0 [139500/178610 (100%)]	Loss: 0.721754

Average loss on epoch 0: 1.0386693600534496



              precision    recall  f1-score   support

           0     0.1289    0.7105    0.2183       380
           1     0.0000    0.0000    0.0000         0
           2     0.9572    0.9695    0.9633     35876
           3     0.2272    0.2223    0.2247      2110
           4     0.8291    0.3981    0.5379      6534

    accuracy                         0.8490     44900
   macro avg     0.4285    0.4601    0.3888     44900
weighted avg     0.8972    0.8490    0.8604     44900

Epoch 1


Train Epoch: 1 [0/178610 (0%)]	Loss: 0.695728
Train Epoch: 1 [35712/178610 (20%)]	Loss: 0.778437
Train Epoch: 1 [71424/178610 (40%)]	Loss: 0.974725
Train Epoch: 1 [107136/178610 (60%)]	Loss: 0.760743
Train Epoch: 1 [142848/178610 (80%)]	Loss: 0.744746
Train Epoch: 1 [139500/178610 (100%)]	Loss: 0.767573

Average loss on epoch 1: 0.7458715805045083



              precision    recall  f1-score   support

           0     0.6686    0.6173    0.6419      2268
           1     0.3291    0.5681    0.4168       734
           2     0.9582    0.9819    0.9699     35460
           3     0.3285    0.3119    0.3200      2174
           4     0.8709    0.6407    0.7383      4264

    accuracy                         0.8919     44900
   macro avg     0.6311    0.6240    0.6174     44900
weighted avg     0.8945    0.8919    0.8908     44900

Epoch 2


...


              precision    recall  f1-score   support

           0     0.8543    0.9279    0.8896      1928
           1     0.7790    0.8379    0.8074      1178
           2     0.9867    0.9914    0.9890     36164
           3     0.7922    0.7177    0.7531      2278
           4     0.9452    0.8845    0.9139      3352

    accuracy                         0.9628     44900
   macro avg     0.8715    0.8719    0.8706     44900
weighted avg     0.9626    0.9628    0.9624     44900

Epoch 13


Train Epoch: 13 [0/178610 (0%)]	Loss: 0.034391
Train Epoch: 13 [35712/178610 (20%)]	Loss: 0.188870
Train Epoch: 13 [71424/178610 (40%)]	Loss: 0.178040
Train Epoch: 13 [107136/178610 (60%)]	Loss: 0.090314
Train Epoch: 13 [142848/178610 (80%)]	Loss: 0.159508
Train Epoch: 13 [139500/178610 (100%)]	Loss: 0.118112

Average loss on epoch 13: 0.13494139134387353



              precision    recall  f1-score   support

           0     0.8381    0.9340    0.8835      1879
           1     0.7569    0.8766    0.8124      1094
           2     0.9874    0.9910    0.9892     36205
           3     0.8178    0.6912    0.7492      2442
           4     0.9302    0.8896    0.9095      3280

    accuracy                         0.9621     44900
   macro avg     0.8661    0.8765    0.8687     44900
weighted avg     0.9621    0.9621    0.9616     44900

Epoch 14


Train Epoch: 14 [0/178610 (0%)]	Loss: 0.170317
Train Epoch: 14 [35712/178610 (20%)]	Loss: 0.104050
Train Epoch: 14 [71424/178610 (40%)]	Loss: 0.052694
Train Epoch: 14 [107136/178610 (60%)]	Loss: 0.083400
Train Epoch: 14 [142848/178610 (80%)]	Loss: 0.080238
Train Epoch: 14 [139500/178610 (100%)]	Loss: 0.216268

Average loss on epoch 14: 0.1188175538080951



              precision    recall  f1-score   support

           0     0.8854    0.9057    0.8954      2047
           1     0.8043    0.8030    0.8036      1269
           2     0.9831    0.9934    0.9883     35961
           3     0.8309    0.6863    0.7517      2499
           4     0.9127    0.9165    0.9146      3124

    accuracy                         0.9616     44900
   macro avg     0.8833    0.8610    0.8707     44900
weighted avg     0.9602    0.9616    0.9605     44900

CPU times: user 1h 25min 13s, sys: 13min 45s, total: 1h 38min 58s
Wall time: 31min 3s


In [46]:
ffnet.eval()
with torch.no_grad():
    preds = []
    gt = []
    for batch, (batch_X, batch_Y) in enumerate(tqdm(test_loader)):
        # Transfer to GPU
        batch_X = batch_X.float().to(device)
        output = nn.Softmax(dim=1)(ffnet(batch_X))
        preds.append(output.cpu())
        gt.append(batch_Y)

    all_out = [np.argmax(l) for batch in preds for l in batch.numpy()]
    all_gt  = [l for batch in gt for l in batch.numpy()]

    print(classification_report(all_out, all_gt, digits=4))


              precision    recall  f1-score   support

           0     0.8208    0.8333    0.8270      1896
           1     0.7527    0.6544    0.7001      1056
           2     0.9708    0.9930    0.9818     31921
           3     0.7766    0.6136    0.6856      3155
           4     0.8550    0.8679    0.8614      2732

    accuracy                         0.9390     40760
   macro avg     0.8352    0.7924    0.8112     40760
weighted avg     0.9354    0.9390    0.9363     40760

